In [69]:
import torch
import librosa
import pandas as pd
import numpy as np
import pylangacq

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [70]:
train_df_cc = pd.read_csv('datasets/addresso/train/cc_meta_data.csv')
train_df_cd = pd.read_csv('datasets/addresso/train/cd_meta_data.csv')
train_df_cc['path'] = 'datasets/addresso/train/transcription/cc/' + train_df_cc['ID'] + '.cha'
train_df_cd['path'] = 'datasets/addresso/train/transcription/cd/' + train_df_cd['ID'] + '.cha'
train_df_cc['class'] = 0
train_df_cd['class'] = 1

train_df = pd.concat([train_df_cc, train_df_cd], ignore_index=True)

train_df.head()

,ID,age,gender,mmse,path,class
0,S001,74,male,NaN,datasets/addresso/train/transcription/cc/S001.cha,0
1,S002,62,female,30.0,datasets/addresso/train/transcription/cc/S002.cha,0
2,S003,69,female,29.0,datasets/addresso/train/transcription/cc/S003.cha,0
3,S004,71,female,30.0,datasets/addresso/train/transcription/cc/S004.cha,0
4,S005,74,female,30.0,datasets/addresso/train/transcription/cc/S005.cha,0


In [71]:
vocab = []

In [72]:
def path_to_features(path):
    chat = pylangacq.read_chat(path)
    utterances = [utterance for utterance in chat.utterances() if utterance.participant == 'PAR']
    tokens = [token for utterance in utterances for token in utterance.tokens]
    words = [token.word for token in tokens]
    
    # bag of words
    features = np.zeros(1000)
    for word in words:
        if word in vocab:
            features[vocab.index(word)] += 1
        else:
            vocab.append(word)
            features = np.append(features, 1)

    # number of words
    features = np.append(features, len(words))

    return features

In [78]:
train_df['features'] = train_df['path'].apply(path_to_features)

X = np.stack(train_df['path'].apply(path_to_features))
y = train_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:
clf = SVC(kernel='linear')

In [110]:
scores = cross_val_score(clf, X_train, y_train, cv=5)

# Test

In [112]:
print(np.mean(scores))

0.8366013071895424
